In [1]:
# uncomment these two lines if using a gpu for a speedup
#! nvidia-smi
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from desc import set_device
set_device("gpu") 
! nvidia-smi

Fri Nov 15 11:38:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     On  | 00000000:05:00.0 Off |                    0 |
|  0%   32C    P8              21W / 300W |     27MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from desc import equilibrium

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


RuntimeError: Unknown backend: 'gpu' requested, but no platforms that are instances of gpu are present. Platforms present are: cpu

In [ ]:
# imports

import numpy as np
from desc.coils import CoilSet, FourierPlanarCoil
import desc.examples
from desc.equilibrium import Equilibrium
from desc.plotting import plot_surfaces, plot_2d, plot_3d, plot_coils
from desc.grid import LinearGrid, ConcentricGrid
from desc.coils import MixedCoilSet
from desc.objectives import (
    ObjectiveFunction,
    #coil
    CoilCurvature,
    CoilLength,
    CoilTorsion,
    CoilSetMinDistance,
    PlasmaCoilSetMinDistance,
    QuadraticFlux,
    ToroidalFlux,
    FixCoilCurrent,
    FixParameters,
    # plasma
    FixBoundaryR,
    FixBoundaryZ,
    FixPressure,
    FixCurrent,
    FixIota,
    FixPsi,
    AspectRatio,
    ForceBalance,
    QuasisymmetryBoozer,
    QuasisymmetryTwoTerm,
    QuasisymmetryTripleProduct,
    VacuumBoundaryError,
)
from desc.continuation import solve_continuation_automatic
from desc.optimize import Optimizer
from desc.magnetic_fields import field_line_integrate
#from desc.singularities import compute_B_plasma
import time
import plotly.express as px
import plotly.io as pio

from desc.geometry import FourierRZToroidalSurface


# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# plotting
from desc.plotting import (
    plot_grid,
    plot_boozer_modes,
    plot_boozer_surface,
    plot_qs_error,
    plot_boundaries,
    plot_boundary,
)

In [ ]:

##################################
# Generating initial equilibrium #
##################################

eq_from_file = desc.io.load("qs_initial_guess.h5")
surf = eq_from_file.surface
eq_init = Equilibrium(Psi=0.04, surface=surf)
print(eq_init.c_l)
eq_init.solve()
# # create initial surface. Aspect ratio ~ 8, circular cross section with slight
# # axis torsion to make it nonplanar
# surf = FourierRZToroidalSurface(
#     R_lmn=[1, 0.125, 0.1],
#     Z_lmn=[-0.125, -0.1],
#     modes_R=[[0, 0], [1, 0], [0, 1]],
#     modes_Z=[[-1, 0], [0, -1]],
#     NFP=2,
# )
# # create initial equilibrium. Psi chosen to give B ~ 1 T. Could also give profiles here,
# # default is zero pressure and zero current
# eq_init = Equilibrium(M=2, N=2, Psi=0.04, surface=surf)


##############################
# Generating initial coilset #
##############################

minor_radius = eq_init.compute("a")["a"]
offset = 0.3
num_coils = 4  # coils per half field period

zeta = np.linspace(0, np.pi / eq_init.NFP, num_coils, endpoint=False) + np.pi / (
    2 * eq_init.NFP * num_coils
)
grid = LinearGrid(rho=[0.0], M=0, zeta=zeta, NFP=eq_init.NFP)
data = eq_init.axis.compute(["x", "x_s"], grid=grid, basis="rpz")

centers = data["x"]  # center coils on axis position
normals = data["x_s"]  # make normal to coil align with tangent along axis

unique_coils = []
for k in range(num_coils):
    coil = FourierPlanarCoil(
        current=1e6,
        center=centers[k, :],
        normal=normals[k, :],
        r_n=minor_radius + offset,
        basis="rpz",  # we are giving the center and normal in cylindrical coordinates
    ).to_FourierXYZ(
        N=15
    )  # fit with 10 fourier coefficients per coil
    unique_coils.append(coil)

# We package these coils together into a CoilSet, which has efficient methods for calculating
# the total field while accounting for field period and stellarator symmetry
# Note that `CoilSet` requires all the member coils to have the same parameterization and resolution.
# if we wanted to use coils of different types or resolutions, we can use a `MixedCoilSet` (see the next section below)
coilset = CoilSet(unique_coils, NFP=eq_init.NFP, sym=eq_init.sym)

In [ ]:
fig = plot_3d(eq_init,"|B|")
plot_coils(coilset,fig=fig)

In [ ]:
# #####################################################################################
# # visualize the initial coilset

# # we use a smaller than usual plot grid to reduce memory of the notebook file
# plot_grid = LinearGrid(M=20, N=40, NFP=1, endpoint=True)
# fig = plot_3d(eq_init, "|B|", grid=plot_grid)
# fig = plot_coils(coilset, fig=fig)
# fig.show()


In [ ]:
grid_vol = ConcentricGrid(
    L=eq_init.L_grid,
    M=eq_init.M_grid,
    N=eq_init.N_grid,
    NFP=eq_init.NFP,
    sym=eq_init.sym,
)

grid_lcfs = LinearGrid(
    M=eq_init.M_grid,
    N=eq_init.N_grid,
    NFP=eq_init.NFP,
    sym=eq_init.sym,
    rho=np.array(1.0),
)

coil_grid = LinearGrid(N=50)
plasma_grid = LinearGrid(M=100, N=300, NFP=eq_init.NFP, sym=eq_init.sym)

coil_indices_to_fix_current = [False for c in coilset]
coil_indices_to_fix_current[0] = True
constraints = (
    ForceBalance(eq=eq_init),
    FixCoilCurrent(coilset, indices=coil_indices_to_fix_current),
    FixPressure(eq=eq_init),  # fix pressure profile
    #FixIota(eq=eq_init),  # fix rotational transform profile
    FixPsi(eq=eq_init),  # fix total toroidal magnetic flux
    )

obj = ObjectiveFunction(
    (

        # QuasisymmetryBoozer(
        #     eq = eq_init, 
        #     #grid = grid_vol, 
        #     helicity = (1, -eq_init.NFP),
        #     weight = 50
        # ),
        QuasisymmetryTwoTerm(
            eq = eq_init, 
            helicity=(1, -eq_init.NFP), 
            grid = grid_lcfs,
            weight = 20
        ),
        # QuasisymmetryTripleProduct(
        #     eq = eq_init, 
        #     grid = grid_vol,
        #     weight = 20
        # ),
        VacuumBoundaryError(
            eq = eq_init,
            field = coilset,
            weight = 100
        ),
         CoilSetMinDistance(
             coilset,
             # in normalized units, want coil-coil distance to be at least 10% of minor radius
             bounds=(0.1, np.inf),
             normalize_target=False,  # we're giving bounds in normalized units
             grid=coil_grid,
             weight=100,
         ),
         PlasmaCoilSetMinDistance(
             eq_init,
             coilset,
             # in normalized units, want plasma-coil distance to be at least 25% of minor radius
             bounds=(0.30, np.inf),
             normalize_target=False,  # we're giving bounds in normalized units
             plasma_grid=plasma_grid,
             coil_grid=coil_grid,
             eq_fixed=False,  # Fix the equilibrium. For single stage optimization, this would be False
             weight=200,
         ),
        CoilCurvature(
            coilset,
            # this uses signed curvature, depending on whether it curves towards
            # or away from the centroid of the curve, with a circle having positive curvature.
            # We give the bounds normalized units, curvature of approx 1 means circular,
            # so we allow them to be a bit more strongly shaped
            bounds=(-1, 2),
            normalize_target=False,  # we're giving bounds in normalized units
            grid=coil_grid,
            weight=200,
        ),
        CoilLength(
             coilset,
             bounds=(0, 2 * np.pi * (minor_radius + offset)),
             normalize_target=True,  # target length is in meters, not normalized
             grid=coil_grid,
             weight=100,
        ),
    )
)


# def compute_average_normalized_field(field, eq_init, vacuum=False):
#     grid = LinearGrid(M=80, N=80, NFP=eq_init.NFP)
#     Bn, surf_coords = field.compute_Bnormal(eq_init, eval_grid=grid)
#     normalizing_field_vec = field.compute_magnetic_field(surf_coords)
#     if not vacuum:
#         # add plasma field to the normalizing field
#         normalizing_field_vec += compute_B_plasma(eq_init, eval_grid=grid)
#     normalizing_field = np.mean(np.linalg.norm(normalizing_field_vec, axis=1))
#     return np.mean(np.abs(Bn)) / normalizing_field


# def plot_field_lines(field, eq_init):
#     # for starting locations we'll pick positions on flux surfaces on the outboard midplane
#     grid_trace = LinearGrid(rho=np.linspace(0, 1, 9))
#     r0 = eq_init.compute("R", grid=grid_trace)["R"]
#     z0 = eq_init.compute("Z", grid=grid_trace)["Z"]
#     fig, ax = desc.plotting.plot_surfaces(eq_init)
#     fig, ax = desc.plotting.poincare_plot(
#         field,
#         r0,
#         z0,
#         NFP=eq_init.NFP,
#         ax=ax,
#         color="k",
#         size=1,
#     )
#     return fig, ax

In [ ]:
optimizer = Optimizer("proximal-lsq-exact")


(eq_qs_T, optimized_coilset,), result_T = optimizer.optimize(
    (eq_init, coilset), 
    objective=obj,
    constraints=constraints,
    ftol=1e-8,  # stopping tolerance on the function value
    xtol=1e-8,  # stopping tolerance on the step size
    gtol=1e-8,  # stopping tolerance on the gradient
    maxiter=1000,  # maximum number of iterations
    options={
        "perturb_options": {"order": 2, "verbose": 0},  # use 2nd-order perturbations
        "solve_options": {
            "ftol": 5e-9,
            "xtol": 5e-5,
            "gtol": 1e-6,
            "verbose": 0,
        },  # for equilibrium subproblem
    },
    copy=True,  # copy=True to make a copy of the optimized result and dont touch the orignals
    verbose=3,
)


In [ ]:
plot_boozer_surface(eq_init);
plot_boozer_surface(eq_qs_T);  # |B| contours at rho=1 surface

In [ ]:
# compare f_T & f_C before (o) vs after (x) optimization
fig, ax = plot_qs_error(
    eq_init, helicity=(1, eq_init.NFP), fB=False, legend=False, rho=10
)
plot_qs_error(
    eq_qs_T, helicity=(1, eq_qs_T.NFP), fB=False, ax=ax, marker=["x", "x"], rho=10
);

In [ ]:
# compare f_B before (o) vs after (x) optimization
fig, ax = plot_qs_error(
    eq_init, helicity=(1, eq_init.NFP), fT=False, fC=False, legend=False, rho=10
)
plot_qs_error(
    eq_qs_T, helicity=(1, eq_qs_T.NFP), fT=False, fC=False, ax=ax, marker=["x"], rho=10
);

In [ ]:
fig=plot_3d(eq_qs_T.surface,"B*n",field=optimized_coilset) # pass in surface so the Bplasma is not computed, as it is vacuum
plot_coils(optimized_coilset,fig=fig)
#plot_coils(coilset,fig=fig,color="red")
